In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from IPython.display import Markdown, display, HTML

load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

df = pd.read_parquet('./data/splade.parquet')
print(f"df shape: {df.shape}")
df.head(1)

df shape: (1987, 24)


,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context,context_citation,context_tokens,openai_embeddings,splade_embeddings,state
0,0,411690,154 Ill. 2d 90,"RICHARD R. JOHNSON, Plaintiff-Appellant and Cr...",Johnson v. Halloran,2000-01-13,8837,Illinois Appellate Court,ill-app-ct,[],"['Wolter, Beeman, Lynch & McIntyre, of Springf...","[{'type': 'official', 'cite': '312 Ill. App. 3...",https://api.case.law/v1/cases/411690/,"RICHARD R. JOHNSON, Plaintiff-Appellant and Cr...",JUSTICE HALL\r\ndelivered the opinion of the c...,False,True,2000,The public defender of Cook County was appoint...,154 Ill. 2d 90,1317,"[-0.0017778094625100493, -0.002360282698646187...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",New Mexico


In [2]:
query = """
How have courts interpreted the phrase 'sudden and accidental' with respect to water damage?
"""

In [3]:
from src.search.models import (
    QueryPlanConfig,
    SearchRequest,
    DocumentSource,
    BaseFilters,
    Tag
)
from src.search.query_expansion import segment_search_query
from src.search.query_planning import generate_query_filters, screen_query

DEFAULT_TOP_K = 40
DEFAULT_RERANK_TOP_K = 20
DEFAULT_MAX_LLM_CHUNKS = 10


def retrieval_preprocessing(
    search_request: SearchRequest,
    verbose: bool = False,
) -> QueryPlanConfig:
    
    tag_list = []
    
    query = search_request.query

    preset_filters = search_request.human_selected_filters or None
    
    if preset_filters:
        tag_list.extend(preset_filters.tags)

    query_screening = None
    if search_request.enable_query_screening:
        query_screening = screen_query(query=query)
        
    if search_request.enable_auto_detect_filters:
        predicted_filters = generate_query_filters(
            search_df=df,
            query=query,
            filter_fields=['state'],
            verbose=verbose,
        )
        pred_tags = []
        for tag in predicted_filters:
            pred_tag = Tag(
                tag_key=tag.filter_key,
                tag_value=tag.filter_value
            )
            pred_tags.append(pred_tag)
        
        tag_list.extend(pred_tags)
        
    if search_request.enable_auto_subquestions:
        n = str(query_screening.n_subquestions) if query_screening else "2 to 3"
        queries = segment_search_query(
            query,
            n,
        )
        
    return QueryPlanConfig(
        query=search_request.query,
        subquestions=queries,
        filters=tag_list,
        num_hits=DEFAULT_TOP_K,
        num_rerank=DEFAULT_RERANK_TOP_K,
        max_llm_filter_chunks=DEFAULT_MAX_LLM_CHUNKS,
        rerank=search_request.rerank,
        llm_chunk_filter=search_request.llm_chunk_filter,
        query_eval=query_screening if query_screening is not None else None,
        search_configs=search_request.search_configs,
    )


In [4]:
from src.agent.tools.semantic_search import SemanticSearchConfig
from src.agent.tools.splade_search import SPLADESearchConfig

DATA_PATH = "data/splade.parquet"

SEMANTIC_TEXT_COLUMN = "context"
SPLADE_TEXT_COLUMN = "context"

semantic_config = SemanticSearchConfig(
    data_path=DATA_PATH,
    text_column=SEMANTIC_TEXT_COLUMN,
)
keyword_config = SPLADESearchConfig(
    data_path=DATA_PATH,
    text_column=SEMANTIC_TEXT_COLUMN,
)

# Tag can be added manually &/or be generated by llm
state_tag = Tag(
    tag_key="state",
    tag_value="Texas"
)
base_filters = BaseFilters(
    source_type=DocumentSource.OPINIONS,
    document_set="sharepoint",
    tags=[state_tag],
)
search_request = SearchRequest(
    query=query,
    human_selected_filters=None,
    enable_query_screening=True,
    enable_auto_detect_filters=True,
    enable_auto_subquestions=True,
    rerank=False,
    llm_chunk_filter=False,
    search_configs=[semantic_config, keyword_config]
)
preprocessed_search_query = retrieval_preprocessing(
    search_request=search_request,
    verbose=True,
)

2024-04-29 00:41:22 - INFO - Schema shown to LLM: 
Name of each field, its type and unique values (up to 20):
* state (string);  Values - ['New Mexico' 'Nevada' 'Virginia' 'Vermont' 'Kansas' 'Maryland' 'Arkansas'
 'Massachusetts' 'West Virginia' 'Pennsylvania' 'Texas' 'Maine'
 'Mississippi' 'Hawaii' 'Iowa' 'Kentucky' 'Ohio' 'New Hampshire'
 'Wisconsin' 'North Dakota'], ... 30 more
        


In [5]:
preprocessed_search_query.model_dump()

{'query': "\nHow have courts interpreted the phrase 'sudden and accidental' with respect to water damage?\n",
 'filters': [{'tag_key': 'text',
   'tag_value': 'sudden and accidental water damage'}],
 'num_hits': 40,
 'rerank': False,
 'num_rerank': 20,
 'llm_chunk_filter': False,
 'max_llm_filter_chunks': 10,
 'subquestions': {'questions': [{'sub_question_topic': "Court Interpretations of 'Sudden and Accidental' in Water Damage Claims",
    'sub_question_query': "How have courts interpreted the phrase 'sudden and accidental' in the context of water damage claims, and what are the key legal precedents and judicial decisions that have influenced this interpretation?",
    'sub_question_keywords': ['sudden and accidental',
     'water damage',
     'court interpretation',
     'legal precedent',
     'judicial decision']}]},
 'query_eval': {'topic': "Interpretation of 'sudden and accidental' phrase in water damage cases",
  'question_quality': 'The question is well-defined and focused on 

In [6]:
preprocessed_search_query.filters[0].filter_dict

{'text': 'sudden and accidental water damage'}

In [7]:
preprocessed_search_query.subquestions.questions[0].sub_question_query

"How have courts interpreted the phrase 'sudden and accidental' in the context of water damage claims, and what are the key legal precedents and judicial decisions that have influenced this interpretation?"

In [8]:
preprocessed_search_query.subquestions.questions[0].sub_question_keywords

['sudden and accidental',
 'water damage',
 'court interpretation',
 'legal precedent',
 'judicial decision']

In [9]:
retrieval_res = preprocessed_search_query.subquestions.execute(
    preprocessed_search_query,
)

2024-04-29 00:41:33 - INFO - Read in df with shape: (1987, 24)
2024-04-29 00:41:33 - INFO - Read in df with shape: (1987, 24)
2024-04-29 00:41:34 - INFO - Using pre-computed 'context' embeddings from existing column: splade_embeddings
2024-04-29 00:41:34 - INFO - 

Thought: To understand how courts have interpreted the phrase 'sudden and accidental' in the context of water
damage, we need to explore case law, legal interpretations, and judicial decisions that have
addressed this specific phrase. This involves looking into how various jurisdictions have defined
and applied 'sudden and accidental' in water damage claims, including any significant rulings that
have shaped the understanding of this phrase.
Search topic: Court Interpretations of 'Sudden and Accidental' in Water Damage Claims
2024-04-29 00:41:34 - INFO - Running search using SemanticSearch: How have courts interpreted the phrase 'sudden and accidental' in the context of water damage claims, and what are the key legal precede

In [10]:
retrieval_res[0].to_pandas.head(2)

,context,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context_citation,context_tokens,state,search_type,score
1675,Id. at 4. According to Plaintiff's insurance p...,2126,12526556,91 L. Ed. 2d 265,"Audrey SANKO, Plaintiff, v. ALLSTATE INSURANCE...",Sanko v. Allstate Ins. Co.,2018-05-18,9039,United States District Court for the Middle Di...,md-pa,[],"['Tammy Lee Clause, Attorney Tammy Lee Clause,...","[{'type': 'official', 'cite': '323 F. Supp. 3d...",https://api.case.law/v1/cases/12526556/,"Audrey SANKO, Plaintiff,\r\nv.\r\nALLSTATE INS...","Robert D. Mariani, United States District Judg...",False,True,2018,91 L. Ed. 2d 265,1442,Wyoming,vector,0.968254
584,"In the absence of clear Missouri law, the tria...",754,5454708,197 Ill. 2d 278,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",Emerson Electric Co. v. Aetna Casualty & Suret...,2004-08-30,8837,Illinois Appellate Court,ill-app-ct,"['McNULTY and McBRIDE, JJ., concur.']","['Stephan G. Weil, Mark H. Kolman, Keisha A. G...","[{'type': 'official', 'cite': '352 Ill. App. 3...",https://api.case.law/v1/cases/5454708/,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",JUSTICE GORDON\r\ndelivered the opinion of the...,False,True,2004,197 Ill. 2d 278,1285,Massachusetts,vector,0.962121


In [11]:
retrieval_res[0].metrics[:10]

[ChunkMetric(source='SemanticSearch', document_id='584', score=0.8603, rank=1),
 ChunkMetric(source='SemanticSearch', document_id='1671', score=0.8422, rank=2),
 ChunkMetric(source='SemanticSearch', document_id='1675', score=0.842, rank=3),
 ChunkMetric(source='SemanticSearch', document_id='1670', score=0.8412, rank=4),
 ChunkMetric(source='SemanticSearch', document_id='1673', score=0.8409, rank=5),
 ChunkMetric(source='SemanticSearch', document_id='1672', score=0.8407, rank=6),
 ChunkMetric(source='SemanticSearch', document_id='1674', score=0.8407, rank=7),
 ChunkMetric(source='SemanticSearch', document_id='585', score=0.8372, rank=8),
 ChunkMetric(source='SemanticSearch', document_id='1955', score=0.8369, rank=9),
 ChunkMetric(source='SemanticSearch', document_id='887', score=0.8359, rank=10)]

In [12]:
from src.agent.tools.utils import aget_fact_patterns_df, aget_context_evaluations_df
import nest_asyncio
nest_asyncio.apply()

rag_df = retrieval_res[0].to_pandas.drop_duplicates(subset=['name'], keep='first')

prepped_context = await aget_fact_patterns_df(rag_df, 'context')

Processing summaries:   0%|          | 0/9 [00:00<?, ?it/s]

Processing summaries: 100%|██████████| 9/9 [00:10<00:00,  1.21s/it]


In [13]:
prepped_context = await aget_context_evaluations_df(
    df=prepped_context,
    question=query,
    text_column='context',
    model_name='gpt-4-turbo',    
)
print(prepped_context['relevance_score'].tolist()[0])

Processing context evaluations: 100%|██████████| 9/9 [00:24<00:00,  2.70s/it]

9


In [14]:
i = 0

display(Markdown(str(prepped_context['relevance_score'].tolist()[i])))
display(Markdown(prepped_context['summary_analysis'].tolist()[i]))

9

The excerpt discusses the interpretation of the terms 'sudden and accidental' in the context of insurance claims and environmental law. It specifically mentions that in Illinois law, as argued by plaintiffs, these terms mean 'unexpected and unintended'. This interpretation, however, was in contention with Missouri law, which the trial court applied, interpreting 'sudden' to mean 'abrupt'. This legal dispute centers on whether the trial court should have considered the broader, Illinois interpretation, especially in light of the abrupt and unintended releases of pollutants at specific sites in Alabama and Louisiana, possibly affecting contamination assessments. The excerpt also delves into procedural aspects like summary judgment and the law of the case doctrine, indicating how interpretations can vary and affect legal outcomes in environmental or contamination-related insurance claims.

In [15]:
Markdown(prepped_context['summary'].tolist()[i])

The central issues in this legal dispute involve the interpretation of insurance policy language, specifically the terms "sudden and accidental" in the context of environmental contamination at owned sites in Alabama and Louisiana. On appeal, plaintiffs argue that the trial court erroneously applied Missouri law instead of Illinois law, the latter of which interprets "sudden and accidental" to mean "unexpected and unintended." Moreover, plaintiffs contest the trial court's interpretation of "sudden" as "abrupt" under Missouri law and its disregard of evidence indicating abrupt and unintended pollutant releases could have contributed to the contamination. The procedural backdrop includes a summary judgment framework, which dictates that such a judgment is appropriate only when there's no genuine issue of material fact and the moving party is entitled to judgment as a matter of law. The order granting summary judgment is subject to de novo review, with appellate courts required to independently examine the evidence for and against the motion to ensure that reasonable divergent inferences from undisputed facts could not lead to a different conclusion. Additionally, this case touches upon the law of the case doctrine, emphasizing its role in ensuring consistency and finality in litigation, albeit with an acknowledgment of its flexibility under certain circumstances like intervening changes in law. The previous decision in Emerson I comes under scrutiny for its interpretation relevant to the pollution exclusion exception in insurance policies, particularly the defining moment of "discharge" or "release" that triggers said exception.

In [16]:
prepped_context.head(3)

,context,index,id,citation,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context_citation,context_tokens,state,search_type,score,summary,summary_analysis,relevance_score
1,"In the absence of clear Missouri law, the tria...",754,5454708,197 Ill. 2d 278,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",Emerson Electric Co. v. Aetna Casualty & Suret...,2004-08-30,8837,Illinois Appellate Court,ill-app-ct,"['McNULTY and McBRIDE, JJ., concur.']","['Stephan G. Weil, Mark H. Kolman, Keisha A. G...","[{'type': 'official', 'cite': '352 Ill. App. 3...",https://api.case.law/v1/cases/5454708/,"EMERSON ELECTRIC COMPANY et al., Plaintiffs-Ap...",JUSTICE GORDON\r\ndelivered the opinion of the...,False,True,2004,197 Ill. 2d 278,1285,Massachusetts,vector,0.962121,The central issues in this legal dispute invol...,The excerpt discusses the interpretation of th...,9
0,Id. at 4. According to Plaintiff's insurance p...,2126,12526556,91 L. Ed. 2d 265,"Audrey SANKO, Plaintiff, v. ALLSTATE INSURANCE...",Sanko v. Allstate Ins. Co.,2018-05-18,9039,United States District Court for the Middle Di...,md-pa,[],"['Tammy Lee Clause, Attorney Tammy Lee Clause,...","[{'type': 'official', 'cite': '323 F. Supp. 3d...",https://api.case.law/v1/cases/12526556/,"Audrey SANKO, Plaintiff,\r\nv.\r\nALLSTATE INS...","Robert D. Mariani, United States District Judg...",False,True,2018,91 L. Ed. 2d 265,1442,Wyoming,vector,0.968254,The insurance policy's exclusions for coverage...,The excerpt deals extensively with coverage di...,8
7,Varlen found a chemical called hexavalent chro...,2614,12521187,91 L. Ed. 2d 202,"VARLEN CORPORATION, Plaintiff-Appellant, v. LI...",Varlen Corp. v. Liberty Mut. Ins. Co.,2019-05-16,9010,United States Court of Appeals for the Seventh...,7th-cir,"['Before Sykes, Barrett, and St. Eve, Circuit ...","['Catherine Basque Weiler, Attorney, Swanson, ...","[{'type': 'official', 'cite': '924 F.3d 456'}]",https://api.case.law/v1/cases/12521187/,"VARLEN CORPORATION, Plaintiff-Appellant,\r\nv....","Barrett, Circuit Judge.\r\nVarlen Corporation ...",True,True,2019,91 L. Ed. 2d 202,1339,Minnesota,vector,0.678113,In a legal dispute between Varlen and its insu...,The excerpt details a legal conflict between V...,8


In [17]:
from src.agent.tools.utils import create_formatted_input, get_final_answer

formatted_input = create_formatted_input(
    df=prepped_context, 
    query=retrieval_res[0].sub_question, 
    text_column='summary',
    url_column='url',
    context_token_limit=6000
    )

In [18]:
Markdown(formatted_input)

[1] The central issues in this legal dispute involve the interpretation of insurance policy language, specifically the terms "sudden and accidental" in the context of environmental contamination at owned sites in Alabama and Louisiana. On appeal, plaintiffs argue that the trial court erroneously applied Missouri law instead of Illinois law, the latter of which interprets "sudden and accidental" to mean "unexpected and unintended." Moreover, plaintiffs contest the trial court's interpretation of "sudden" as "abrupt" under Missouri law and its disregard of evidence indicating abrupt and unintended pollutant releases could have contributed to the contamination. The procedural backdrop includes a summary judgment framework, which dictates that such a judgment is appropriate only when there's no genuine issue of material fact and the moving party is entitled to judgment as a matter of law. The order granting summary judgment is subject to de novo review, with appellate courts required to independently examine the evidence for and against the motion to ensure that reasonable divergent inferences from undisputed facts could not lead to a different conclusion. Additionally, this case touches upon the law of the case doctrine, emphasizing its role in ensuring consistency and finality in litigation, albeit with an acknowledgment of its flexibility under certain circumstances like intervening changes in law. The previous decision in Emerson I comes under scrutiny for its interpretation relevant to the pollution exclusion exception in insurance policies, particularly the defining moment of "discharge" or "release" that triggers said exception.
URL: https://api.case.law/v1/cases/5454708/

[2] The insurance policy's exclusions for coverage specifically include a variety of water damage scenarios, excluding coverage for losses caused by flood, surface water, sewer or drain backups, sump pump overflows, and subsurface water pressure or seepage. However, it does provide coverage for sudden and accidental physical loss due to fire, explosion, or theft resulting from such water-related issues. In a disputed claim, the plaintiff references a specific event where a sewage system explosion caused damage, arguing for coverage under the policy's exception for explosions. The defendant challenges this claim, asserting that the explosion must be a consequence of the excluded water damage, not a preceding event, to qualify for coverage. This interpretation dispute is set against the backdrop of summary judgment standards emphasizing the need for genuine disputes over material facts to oppose summary judgment effectively. Colella v. State Farm Fire & Cas. Co. is cited as a relevant case for interpreting similar water damage exclusion clauses.
URL: https://api.case.law/v1/cases/12526556/

[3] In a legal dispute between Varlen and its insurer, Liberty Mutual, regarding coverage for contamination damage at the LASI and Silvis sites, the core issue revolved around whether the chemical leaks or discharges were "sudden and accidental" as outlined in Liberty Mutual's policy. Varlen's primary evidence to support this claim was the expert testimony of geologist Daniel Rogers. Rogers posited that the contamination was sudden and accidental based on the characteristics of the concrete sump leak at the LASI site and the nature of diesel and solvent contamination at the Silvis site. However, the district court ruled Rogers's testimony as unreliable and speculative, not meeting the Federal Rule of Evidence 702 standards for expert testimony. This was based on the reasoning that Rogers's conclusions were not sufficiently grounded in reliable methods or principles. Consequently, the court granted Liberty Mutual's motion for summary judgment after striking Rogers's testimony, pointing out that Varlen failed to provide admissible evidence that a reasonable jury could rely on to find the contamination events were "sudden and accidental".
URL: https://api.case.law/v1/cases/12521187/

[4] The case discusses the interpretation of terms within a Commercial General Liability (CGL) policy, specifically focusing on 'property damage' and 'occurrence,' and their implications for coverage. 'Property damage' is defined within the policy as either physical injury to tangible property or loss of use of tangible property that is not physically injured. An 'occurrence' is defined as an accident, which includes continuous or repeated exposure to substantially the same general harmful conditions. However, the policy lacks a definition for 'accident,' leading to ambiguity. Courts have therefore emphasized a liberal construction of 'accident' in favor of coverage, interpreting it as an unforeseen and unplanned event, or an event resulting from carelessness or ignorance where the person responsible did not intend or expect the resulting harm. The specific case highlights a scenario where the plaintiffs allege physical damage to their property due to negligent actions, claiming this damage falls under the policy's broad definition of 'property damage.' The court must decide whether this damage resulted from an 'occurrence' as per the policy's definition. Early arguments suggest that the negligence leading to property damage constitutes an 'occurrence' because the actions leading to the damage were not intended or expected by the defendant, aligning with the judicial interpretation of an 'accident.' The text also touches upon the potential for the CGL policy to be overly broad, suggesting it might cover scenarios typically excluded, akin to a performance bond.
URL: https://api.case.law/v1/cases/4268691/

[5] The plaintiff in a declaratory judgment action bears the burden of proof, as established by Addison Insurance Co., emphasizing the appellees' initial responsibility in such cases. The dispute centers on whether the damage to the Walskis’ home falls under policy coverage, challenged on grounds it was not caused by an “occurrence” and that contractual liability for economic damages does not equate to “property damage.” The policy definition of “occurrence” as an accident or continuous exposure to harmful conditions is scrutinized. Various Illinois appellate court decisions and definitions from Black’s Law Dictionary delineate “accident” as unforeseen, injurious occurrences not expected in the usual course of events. The legal dialogue delves into whether the natural and ordinary consequences of an act, or actions intentionally done with unintended results, constitute an accident. Carr versus Stoneridge cases illuminate differing interpretations based on the nature of claims (negligence versus breach of contract), influenced by precedent from Supreme Court and appellate court rulings, including nuanced discussions on the intent behind actions leading to accidents. The complication around what defines an “accident” underscores the nuanced legal interpretations of policy language and the implications for coverage in cases of property damage claims.
URL: https://api.case.law/v1/cases/4277921/

[6] Visalia's legal battle revolves around disputes concerning environmental contaminant cleanup costs, highlighting the non-relevance of ‘sudden and accidental’ releases due to explicit policy exclusions. Visalia’s reliance on affirmative defenses against RLI, including waiver, estoppel, and fraud, is further nuanced by the addition of 'unclean hands' and 'forfeiture' defenses in their Amended Answer. The parties debated on the appropriate standard for pleading affirmative defenses, referencing a lack of clarity between the 'fair notice' and the heightened 'plausibility' standards established by federal case law. The court dismissed certain requests for judicial notice as irrelevant, underscoring the focus on the legal tenability and the strategic approaches taken by both parties within the litigation framework. The discussion also dabbles in the procedural postings and the specific legal precedents guiding the pleading standards for affirmative defenses within the Ninth Circuit jurisdiction, reflecting broader implications on case strategy and judicial discretion.
URL: https://api.case.law/v1/cases/12524005/

[7] Williams, 143 Ill. App. 3d at 111 involves a dispute over insurance coverage for water damage to property. The core issue revolves around interpreting the policy to determine coverage eligibility. Critical to the case are factual findings regarding the cause of the water damage, with the court finding it resulted from rain collecting on the roof leading to a partial collapse. This finding contrasts with testimonies suggesting the damage was due to normal wear and tear and that no roof repairs had been observed or conducted during the relevant storage period. Notably, the admission of roof repairs between July 1997 and November 1998 supported the court’s decision. The case was marked by conflicting evidence, necessitating multiple factual findings, specifically 15, compared to only three conclusions of law. The appellate court found the circuit court’s order to strike the defendant’s jury demand as an abuse of discretion, emphasizing the need for a jury to evaluate witness credibility and factual causes of the water damage for proper policy coverage determination. Consequently, it was remanded for a new trial. The appellate review also addressed the propriety of denying the defendant's motion for summary judgment, highlighting the existence of factual disputes and criticizing the defendant's contradictory stances on the presence of factual issues. The decision underscores principles of judgment and jury involvement in the context of insurance dispute resolution and factual ambiguity.
URL: https://api.case.law/v1/cases/4267133/

[8] Hugo's Estate argues for a broad interpretation of "occurrence" under Kentucky law to include unintentional injuries from negligent acts, like exposure to secondhand smoke, advocating that such mishaps not planned or known to cause harm should be considered accidents covered by insurance. It disputes the claim of "bodily injury" arising from "physical abuse" as defined in the policy, suggesting that the common understanding of abuse does not align with Hugo's unknowingly harmful smoking around his children. The legal backdrop includes the summary judgment standard, guiding the court to resolve cases without a trial when there's no factual dispute and one party is entitled to judgment as a legal right. Crucially, Kentucky law, chosen based on the most significant relationship to the case -- particularly insurance matters -- emphasizes that insurance contracts are interpreted by the court, focusing on the mutual understanding of the contract terms at inception. Insurers have a duty to defend if claims could potentially fall under policy coverage unless clearly excluded. The text suggests a heavy reliance on statutory interpretation, precedent, and the specific language of the insurance contract in determining the obligations of the insurer and the rights of the insured.
URL: https://api.case.law/v1/cases/12531212/

[9] A legal dispute arose when plaintiffs filed a claim with Union for water damage to their basement, caused by flooding, which Union denied citing a policy exclusion for loss caused by water damage, including floods and surface water. Union argued the damage resulted from flood and surface water, while the plaintiffs contended the water was not surface water and accused Union of shifting its defense inappropriately under the "mend the hold" doctrine. The trial court granted summary judgment in favor of Union, rejecting plaintiffs' arguments, including their claim under the "mend the hold" doctrine, which prevents a party from changing the reason for breach of contract once litigation has begun. The trial court's decision, based on the pleaded water damage exclusion and expert testimonies on the cause of the damage, highlighted the non-applicability of the "mend the hold" doctrine since Union did not change the foundational basis of its defense but rather specified a type of water contributing to the damage. This case underscores the necessity of a clear understanding of insurance policy exclusions and the challenges in litigating insurance claims involving nuanced definitions of excluded perils.
URL: https://api.case.law/v1/cases/171775/

Instructions: Using only the provided search results that are relevant, and starting with the most relevant, write a detailed comparative analysis for a new query. If there are no relevant cases say so, and use one example from the search results to illustrate the uniqueness of the new query. ALWAYS cite search results using [[number](URL)] notation after the reference.

New Query:
How have courts interpreted the phrase 'sudden and accidental' in the context of water damage claims, and what are the key legal precedents and judicial decisions that have influenced this interpretation?

Analysis:

In [19]:
report = get_final_answer(
    formatted_input=formatted_input,
    model_name='gpt-4-turbo',
    )
Markdown(report.research_report)

The phrase 'sudden and accidental' has been a central issue in several insurance policy disputes concerning environmental and other types of damage. Various cases provide insights into how courts interpret this phrase, particularly in the context of water damage claims. 

In the first case, the interpretation of 'sudden and accidental' is disputed in the context of environmental contamination, where plaintiffs argue that under Illinois law, the term should mean 'unexpected and unintended,' contrasting with 'abrupt' under Missouri law [[1](https://api.case.law/v1/cases/5454708/)]. This ongoing debate over the jurisdictional application and definitions underlines the significant difference in interpretations that can affect the outcome of litigation involving insurance coverage for environmental damages.

Another relevant case involves a dispute over whether a sewage system explosion, falling under the exception for explosions despite prior exclusions for water damage, should be considered 'sudden and accidental' [[2](https://api.case.law/v1/cases/12526556/)]. This case illustrates the complexity of separating cause and effect in insurance disputes and the importance of clearly understanding policy terms in determining coverage.

In the third relevant case, the reliability of expert testimony was critical in deciding whether chemical leaks were 'sudden and accidental.' The court's decision to disregard the expert's testimony due to its speculative nature and lack of reliable methodologies reflects the scrutiny legal arguments undergo in such disputes [[3](https://api.case.law/v1/cases/12521187/)].

These cases collectively emphasize the varied judicial interpretations of 'sudden and accidental,' which often hinge on specific facts and expert testimonies, as well as the nuanced understanding of policy language. They reveal that 'sudden and accidental' often involves a detailed analysis of the circumstances surrounding the incident and the policy wording that can shift the balance in judicial decisions.

In [20]:
report.citations

['1', '2', '3']

In [3]:
from src.agent.tools.utils import convert_timestamp_to_datetime
from src.utils.citations import create_citation_lookup_table, get_top_citations, get_citation_context

In [4]:
citation_table = create_citation_lookup_table(df, text_column='body', id_column='id')

In [6]:
citation_table.shape

(20102, 2)

In [7]:
top_citations = get_top_citations(df=df, text_column='body', top_n=10)
Markdown(top_citations.to_markdown(index=False))

| Citation          |   Count | link                                                 |
|:------------------|--------:|:-----------------------------------------------------|
| 154 Ill. 2d 90    |    1331 | https://cite.case.law/ill-2d/154/90                  |
| 607 N.E.2d 1204   |     844 | https://cite.case.law/ne2d/607/1204                  |
| 156 Ill. 2d 384   |     443 | https://cite.case.law/ill-2d/156/384                 |
| 477 U.S. 317      |     413 | https://cite.case.law/us/477/317                     |
| 106 S. Ct. 2548   |     385 | https://www.courtlistener.com/c/S.%20Ct./106/2548    |
| 91 L. Ed. 2d 265  |     385 | https://www.courtlistener.com/c/L.%20Ed.%202d/91/265 |
| 144 Ill. 2d 64    |     351 | https://cite.case.law/ill-2d/144/64                  |
| 180 Ill. Dec. 691 |     265 | https://www.courtlistener.com/c/Ill.%20Dec./180/691  |
| 177 Ill. 2d 473   |     249 | https://cite.case.law/ill-2d/177/473                 |
| 213 Ill. 2d 307   |     243 | https://cite.case.law/ill-2d/213/307                 |

In [8]:
from src.utils.citations import filter_frequent_citations, add_citation_context_to_df

frequent_citation_table = filter_frequent_citations(citation_table, 100)
frequent_citation_table.shape

(499, 2)

In [10]:
df2 = frequent_citation_table.merge(df, on='id', how='inner')
df2.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [28]:
top_cites_with_context_df = add_citation_context_to_df(
    df=df2,
    text_column='body',
    citation_column='citation_x',
)

In [13]:
df2.head(1)

,id,citation_x,index,citation_y,name,name_abbreviation,decision_date,court_id,court_name,court_slug,judges,attorneys,citations,url,head,body,name_contains_lm,body_contains_lm,year,context,context_citation,context_tokens,openai_embeddings,splade_embeddings,state
0,411690,154 Ill. 2d 90,0,154 Ill. 2d 90,"RICHARD R. JOHNSON, Plaintiff-Appellant and Cr...",Johnson v. Halloran,2000-01-13,8837,Illinois Appellate Court,ill-app-ct,[],"['Wolter, Beeman, Lynch & McIntyre, of Springf...","[{'type': 'official', 'cite': '312 Ill. App. 3...",https://api.case.law/v1/cases/411690/,"RICHARD R. JOHNSON, Plaintiff-Appellant and Cr...",JUSTICE HALL\r\ndelivered the opinion of the c...,False,True,2000,The public defender of Cook County was appoint...,154 Ill. 2d 90,1317,"[-0.0017778094625100493, -0.002360282698646187...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",New Mexico


In [36]:
citation_context = ""

for _, row in df2.iterrows():
    cite = row['citation']
    text = row['body']
    name = row['name_abbreviation']
    date = row['decision_date']
    cite_context = get_citation_context(
        text=text,
        citation=cite,
        words_after=200,
        words_before=200,
    )
    if cite_context != "":
        citation_context += f"Citation: **{cite}**\n\n"
        citation_context += f"Case Name: {name}, decided on {date}\n\n"
        citation_context += f"Context: {cite_context}\n\n"
        citation_context += "-"*100 + "\n\n"

In [24]:
from src.utils.gen_utils import count_tokens

def create_citation_context(
    df: pd.DataFrame,
    citation_column: str,
    text_column: str,
    name_column: str,
    date_column: str,
    words_before: int = 200,
    words_after: int = 200,
    context_token_limit: int = 3000,
) -> str:
    """Creates a citation context string from a DataFrame based on the provided columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the citation data.
        citation_column (str): The name of the column containing the citation strings.
        text_column (str): The name of the column containing the text data.
        name_column (str): The name of the column containing the case names.
        date_column (str): The name of the column containing the decision dates.
        words_before (int, optional): The number of words to include before the citation. Defaults to 200.
        words_after (int, optional): The number of words to include after the citation. Defaults to 200.
        context_token_limit (int, optional): The maximum number of tokens allowed in the context. Defaults to 3000.

    Returns:
        str: The citation context string created from the DataFrame.
    """
    citation_context = ""
    total_tokens = 0

    for _, row in df.iterrows():
        cite = row[citation_column]
        text = row[text_column]
        name = row[name_column]
        date = row[date_column].strftime("%Y-%m-%d")

        cite_context = get_citation_context(
            text=text,
            citation=cite,
            words_after=words_after,
            words_before=words_before,
        )

        if cite_context != "":
            context_entry = f"Citation: **{cite}**\n\n"
            context_entry += f"Case Name: {name}, decided on {date}\n\n"
            context_entry += f"Context: {cite_context}\n\n"
            context_entry += "-" * 100 + "\n\n"

            entry_tokens = count_tokens(context_entry)
            if total_tokens + entry_tokens > context_token_limit:
                break

            citation_context += context_entry
            total_tokens += entry_tokens

    return citation_context

In [25]:
test = create_citation_context(
    df=df,
    citation_column='citation',
    text_column='body',
    name_column='name_abbreviation',
    date_column='decision_date'
)

In [26]:
Markdown(test)

Citation: **154 Ill. 2d 90**

Case Name: Johnson v. Halloran, decided on 2000-01-13

Context: In the instant case, defendants filed a motion to dismiss plaintiffs complaint based on the statute of limitations, which was denied. Defendants filed a motion for summary judgment on the basis of sovereign immunity, arguing that public defenders are employees of the state and therefore the circuit court lacked subject matter jurisdiction to hear this case because plaintiffs claims must be brought in the Illinois Court of Claims. The circuit court granted defendants’ motion on May 27, 1998. On appeal plaintiff contends that the circuit court erred in granting summary judgment in favor of the defendants on the basis of sovereign immunity because (a) public defenders are not agents or employees of the state, and (b) defendants’ professional duties to their clients do not arise solely as a result of their government employment. On cross-appeal defendants contend that the circuit court erred in finding that plaintiffs complaint was timely filed. DISCUSSION A. Standard of Review In appeals from summary judgment rulings, this court conducts a de novo review. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 607 N.E.2d 1204 (1992). Summary judgment is appropriate only when there are no genuine issues of material fact and the moving party is entitled to judgment as a matter of law. Purtill v. Hess, 111 Ill. 2d 229, 489 N.E.2d 867 (1986). B. Sovereign Immunity Article XIII, section 4, of the Illinois Constitution of 1970 abolished sovereign immunity “[ejxcept as the General Assembly may provide by law.” Ill. Const. 1970, art. XIII, § 4. The legislature enacted the State Lawsuit Immunity Act, which provides in pertinent part: “Except as provided in *** ‘AN ACT to create the Court of Claims ***’, *** the State of Illinois shall not be made a defendant or party in any court.” 745 ILCS 5/1 (West 1996). The Court of Claims Act established the Court of Claims and endowed it with exclusive jurisdiction to hear certain matters, including “[a]ll claims against the State for damages in cases sounding in tort, if a like cause of action would lie against a private person or corporation in a civil suit.” 705 ILCS 505/8(d) (West 1996).

----------------------------------------------------------------------------------------------------

Citation: **111 Ill. 2d 229**

Case Name: Johnson v. Halloran, decided on 2000-01-13

Context: Defendants filed a motion for summary judgment on the basis of sovereign immunity, arguing that public defenders are employees of the state and therefore the circuit court lacked subject matter jurisdiction to hear this case because plaintiffs claims must be brought in the Illinois Court of Claims. The circuit court granted defendants’ motion on May 27, 1998. On appeal plaintiff contends that the circuit court erred in granting summary judgment in favor of the defendants on the basis of sovereign immunity because (a) public defenders are not agents or employees of the state, and (b) defendants’ professional duties to their clients do not arise solely as a result of their government employment. On cross-appeal defendants contend that the circuit court erred in finding that plaintiffs complaint was timely filed. DISCUSSION A. Standard of Review In appeals from summary judgment rulings, this court conducts a de novo review. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 607 N.E.2d 1204 (1992). Summary judgment is appropriate only when there are no genuine issues of material fact and the moving party is entitled to judgment as a matter of law. Purtill v. Hess, 111 Ill. 2d 229, 489 N.E.2d 867 (1986). B. Sovereign Immunity Article XIII, section 4, of the Illinois Constitution of 1970 abolished sovereign immunity “[ejxcept as the General Assembly may provide by law.” Ill. Const. 1970, art. XIII, § 4. The legislature enacted the State Lawsuit Immunity Act, which provides in pertinent part: “Except as provided in *** ‘AN ACT to create the Court of Claims ***’, *** the State of Illinois shall not be made a defendant or party in any court.” 745 ILCS 5/1 (West 1996). The Court of Claims Act established the Court of Claims and endowed it with exclusive jurisdiction to hear certain matters, including “[a]ll claims against the State for damages in cases sounding in tort, if a like cause of action would lie against a private person or corporation in a civil suit.” 705 ILCS 505/8(d) (West 1996). The plaintiff argues that Cook County public defenders are county employees and, therefore, the circuit court erred in applying the doctrine of sovereign immunity. Defendants contend that they are employees of the state so that sovereign immunity would apply.

----------------------------------------------------------------------------------------------------

Citation: **489 N.E.2d 867**

Case Name: Johnson v. Halloran, decided on 2000-01-13

Context: Defendants filed a motion for summary judgment on the basis of sovereign immunity, arguing that public defenders are employees of the state and therefore the circuit court lacked subject matter jurisdiction to hear this case because plaintiffs claims must be brought in the Illinois Court of Claims. The circuit court granted defendants’ motion on May 27, 1998. On appeal plaintiff contends that the circuit court erred in granting summary judgment in favor of the defendants on the basis of sovereign immunity because (a) public defenders are not agents or employees of the state, and (b) defendants’ professional duties to their clients do not arise solely as a result of their government employment. On cross-appeal defendants contend that the circuit court erred in finding that plaintiffs complaint was timely filed. DISCUSSION A. Standard of Review In appeals from summary judgment rulings, this court conducts a de novo review. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 607 N.E.2d 1204 (1992). Summary judgment is appropriate only when there are no genuine issues of material fact and the moving party is entitled to judgment as a matter of law. Purtill v. Hess, 111 Ill. 2d 229, 489 N.E.2d 867 (1986). B. Sovereign Immunity Article XIII, section 4, of the Illinois Constitution of 1970 abolished sovereign immunity “[ejxcept as the General Assembly may provide by law.” Ill. Const. 1970, art. XIII, § 4. The legislature enacted the State Lawsuit Immunity Act, which provides in pertinent part: “Except as provided in *** ‘AN ACT to create the Court of Claims ***’, *** the State of Illinois shall not be made a defendant or party in any court.” 745 ILCS 5/1 (West 1996). The Court of Claims Act established the Court of Claims and endowed it with exclusive jurisdiction to hear certain matters, including “[a]ll claims against the State for damages in cases sounding in tort, if a like cause of action would lie against a private person or corporation in a civil suit.” 705 ILCS 505/8(d) (West 1996). The plaintiff argues that Cook County public defenders are county employees and, therefore, the circuit court erred in applying the doctrine of sovereign immunity. Defendants contend that they are employees of the state so that sovereign immunity would apply.

----------------------------------------------------------------------------------------------------

Citation: **154 Ill. 2d 90**

Case Name: Great American Insurance v. West Bend Mutual Insurance, decided on 2000-01-18

Context: West Bend admitted that Great American sent statements from witnesses but denied the substance of the alleged statements. West Bend also answered that Great American did not accurately summarize the state-merits. Great American provided only transcripts of interviews with the unsworn witnesses, not depositions or affidavits. West Bend admitted the essential allegations of the complaint and stipulated that Great American paid a fair amount for the settlement. Then West Bend moved for summary judgment, arguing that it owed no part of the settlement because liability under the Act could not qualify as liability imputed to Sundance as a result of Szarek’s conduct. The trial court agreed and added that the cause of the accident remained undetermined. Nonetheless, the court held that “[t]he money Great American paid in settlement arose solely from the conduct of its insured[,] Sundance.” The trial court granted West Bend’s motion for summary judgment. This court reviews summary judgments de novo. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102, 607 N.E.2d 1204 (1992). We accord the terms of an insurance policy their plain meanings, and we construe limitations on coverage strongly against the insurer. National Union Fire Insurance Co. v. Glenview Park District, 158 Ill. 2d 116, 122, 632 N.E.2d 1039 (1994). “[C]ourts should be quick to construe the policy in favor of coverage so that the insured is not deprived of a benefit for which the insured paid.” Dash Messenger Service, Inc. v. Hartford Insurance Co., 221 Ill. App. 3d 1007, 1010, 582 N.E.2d 1257 (1991). Great American pled that Szarek’s employees negligently removed the bracing, causing Bass to fall. West Bend does not dispute the adequacy of the allegations to show that Sundance’s liability arose “as a result of” the acts of Szarek, within the meaning of the insurance policy. West Bend contends only that the verb “imputed” adds a distinct, further constraint on the scope of the insurance and the liability here is not “imputed” to Sundance as a result of Szarek’s acts. Liability for negligence may be imputed where “the person to whom the negligence is imputed *** had a legal right to control the action of the person actually negligent.

----------------------------------------------------------------------------------------------------

Citation: **158 Ill. 2d 116**

Case Name: Great American Insurance v. West Bend Mutual Insurance, decided on 2000-01-18

Context: West Bend also answered that Great American did not accurately summarize the state-merits. Great American provided only transcripts of interviews with the unsworn witnesses, not depositions or affidavits. West Bend admitted the essential allegations of the complaint and stipulated that Great American paid a fair amount for the settlement. Then West Bend moved for summary judgment, arguing that it owed no part of the settlement because liability under the Act could not qualify as liability imputed to Sundance as a result of Szarek’s conduct. The trial court agreed and added that the cause of the accident remained undetermined. Nonetheless, the court held that “[t]he money Great American paid in settlement arose solely from the conduct of its insured[,] Sundance.” The trial court granted West Bend’s motion for summary judgment. This court reviews summary judgments de novo. Outboard Marine Corp. v. Liberty Mutual Insurance Co., 154 Ill. 2d 90, 102, 607 N.E.2d 1204 (1992). We accord the terms of an insurance policy their plain meanings, and we construe limitations on coverage strongly against the insurer. National Union Fire Insurance Co. v. Glenview Park District, 158 Ill. 2d 116, 122, 632 N.E.2d 1039 (1994). “[C]ourts should be quick to construe the policy in favor of coverage so that the insured is not deprived of a benefit for which the insured paid.” Dash Messenger Service, Inc. v. Hartford Insurance Co., 221 Ill. App. 3d 1007, 1010, 582 N.E.2d 1257 (1991). Great American pled that Szarek’s employees negligently removed the bracing, causing Bass to fall. West Bend does not dispute the adequacy of the allegations to show that Sundance’s liability arose “as a result of” the acts of Szarek, within the meaning of the insurance policy. West Bend contends only that the verb “imputed” adds a distinct, further constraint on the scope of the insurance and the liability here is not “imputed” to Sundance as a result of Szarek’s acts. Liability for negligence may be imputed where “the person to whom the negligence is imputed *** had a legal right to control the action of the person actually negligent. Negligence in the conduct of another will not be imputed to a party if he did not authorize such conduct, participate therein, or have the right or power to control it.” Johnson v. Turner, 319 Ill.

----------------------------------------------------------------------------------------------------



In [38]:
import instructor
import openai
from tenacity import Retrying, stop_after_attempt, wait_fixed

from src.search.models import CitationInformation

def analyze_citation(
    citation: str, excerpts: str
) -> CitationInformation:
    client = instructor.patch(openai.OpenAI())
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        seed=42,
        response_model=CitationInformation,
        max_retries=Retrying(
            stop=stop_after_attempt(5),
            wait=wait_fixed(1),
        ),
        messages=[
            {
                "role": "system",
                "content": "Your role is to analyze Context, which is previously extracted information about a legal case law citation, and distill down the main takeaways using the correct format. The goal is to support downstream retrieval for users who will be submitting natural language queries to return relevant and useful case citations.",
            },
            {
                "role": "user",
                "content": f"Your task focuses on citation: **{citation}**",
            },
            {   "role": "user", "content": f"Here is the Context, remember to limit the use of named entities especially on the questions: {excerpts}"},
        ],
    )

In [39]:
cite_res = analyze_citation(
    citation='154 Ill. 2d 90',
    excerpts=citation_context
)

In [40]:
print(cite_res)

Citation: 154 Ill. 2d 90
Summary:
In the context of insurance and summary judgment decisions, 154 Ill. 2d 90 serves as a pivotal
authority on several points: it outlines the conditions under which summary judgment is appropriate,
details appellate review standards for these judgments, emphasizes the broad duty to defend over the
narrower duty to indemnify within insurance contracts, and provides guidance on the construction and
interpretation of insurance policies in favor of ensuring coverage. The case law emphasizes a de
novo review in both summary judgment appeals and insurance policy interpretations, and asserts that
factual inferences drawn from undisputed facts can preclude summary judgment when divergent.
Furthermore, it underscores the principle that an insurer's duty to defend is triggered when
allegations even potentially fall within policy coverage, and confronts challenges such as estoppel
claims against granting summary judgment.
Questions:
- What are the conditions under 